# DMU-Net Dataset Generation Notebook

* **Creator:** Jonathan DEKHTIAR
* **Date:** 2017-05-21
<br/><br/>
* **Contact:** [contact@jonathandekhtiar.eu](mailto:contact@jonathandekhtiar.eu)
* **Twitter:** [@born2data](https://twitter.com/born2data)
* **LinkedIn:** [JonathanDEKHTIAR](https://fr.linkedin.com/in/jonathandekhtiar)
* **Personal Website:** [JonathanDEKHTIAR](http://www.jonathandekhtiar.eu)
* **RSS Feed:** [FeedCrunch.io](https://www.feedcrunch.io/@dataradar/)
* **Tech. Blog:** [born2data.com](http://www.born2data.com/)
* **Github:** [DEKHTIARJonathan](https://github.com/DEKHTIARJonathan)
<br/><br/>

```
*************************************************************************
**
** 2017 March 13
**
** In place of a legal notice, here is a blessing:
**
**    May you do good and not evil.
**    May you find forgiveness for yourself and forgive others.
**    May you share freely, never taking more than you give.
**
*************************************************************************
```

## Objectives

This notebook generates a [TFRecords](https://www.tensorflow.org/api_guides/python/python_io#TFRecords_Format_Details) file from the images contained in the "./data/" folder.

This will be used later to retrain an CNN model: [Inception-V3](http://arxiv.org/abs/1512.00567) model developed by Szegedy et al. The model has been Pre-Trained with the [ImageNet](http://www.image-net.org/) dataset allowing a much more accurate result due to the large number of data avaiable in this dataset. We call this a "Transfer Learning".

## 1. Load the necessary libraries and initialise global variables

In [1]:
import os, string, random

import tensorflow as tf
import numpy as np

import matplotlib.pyplot as plt

%matplotlib inline

################################## GLOBAL NOTEBOOK VARS ##################################

NUM_EPOCH          = 1
INPUT_DIRECTORY    = "data"
OUTPUT_DIRECTORY   = "data_augmented"

if not os.path.exists(OUTPUT_DIRECTORY):
    os.makedirs(OUTPUT_DIRECTORY)

############################### RANDOM VALUE GENERATION SEED #############################
SEED               = 666

######################## Model Dependant Parameters - Inception V3 #######################

IMG_HEIGHT         = 299     # This parameter is fixed due to the model used: Inception-V3
IMG_WIDTH          = 299     # This parameter is fixed due to the model used: Inception-V3
IMG_CHANNELS       = 3       # This parameter is fixed due to the model used: Inception-V3
IMG_COLORSPACE     = "RGB"   # This parameter is fixed due to the model used: Inception-V3
IMG_OUTFORMAT      = "JPEG"  # This parameter is fixed due to the model used: Inception-V3

## 2. File Queue and Image Reading Process Definition

### 2.1 Define a queue of all the images in "jpeg" in the specific data folder

Make a queue of file names including all the JPEG images files in the relative image directory.

In [110]:
data_directories = [ name for name in os.listdir(INPUT_DIRECTORY) if os.path.isdir(os.path.join(INPUT_DIRECTORY, name)) ]

png_ext_list  = ["png"]
jpeg_ext_list = ["jpg", "jpeg"]

ext_list = jpeg_ext_list + png_ext_list # = ['jpg', 'jpeg', 'JPG', 'JPEG', 'png', 'PNG']

all_files = [tf.train.match_filenames_once(INPUT_DIRECTORY + "/" + x + "/*."+ext) for x in data_directories for ext in ext_list]

filename_queue = tf.train.string_input_producer(
    tf.concat(all_files,0), # Merge the sub-tensors into one
    num_epochs=NUM_EPOCH,
    seed=SEED,
    shuffle=True
)

### 2.2. Define the image reader

Read an entire image file which is required since they're JPEGs, if the images are too large they could be split in advance to smaller files or use the Fixed reader to split up the file.

In [111]:
image_reader = tf.WholeFileReader()

### 2.3. Read images from the Queue One by One
Read a whole file from the queue, the first returned value in the tuple is the filename which we are ignoring.

In [112]:
image_path, image_file = image_reader.read(filename_queue)

### 2.4. Convert each Image to a Tensor

Decode the image file, this will turn it into a Tensor which we can then use in training. It automatically detect whether the image is ["GIF", "PNG", "JPEG"] and which decoder to use.

In [113]:
def string_length_tf(t):
    return tf.py_func(lambda x: len(x), [t], tf.int32)

In [114]:
path_length = string_length_tf(image_path)
file_extension = tf.substr(image_path, path_length - 3, 3)

file_cond = tf.equal(file_extension, jpeg_ext_list)
file_cond = tf.count_nonzero(file_cond)
file_cond = tf.equal(file_cond, 1) ## 1 => JPEG EXTENSION, 0 => PNG EXTENSION
        
image_tmp      = tf.cond(
                    file_cond, 
                    lambda: tf.image.decode_jpeg(image_file), 
                    lambda: tf.image.decode_png(image_file)
               )

image_resized  = tf.image.resize_images(
                    image_tmp, 
                    tf.stack([IMG_HEIGHT, IMG_WIDTH]), 
                    method=tf.image.ResizeMethod.BICUBIC,
                    align_corners=True
               )

# resize image by bilinear, bicubic and area will change image data type(from uint8 to float32)
image_data = tf.cast(image_resized, tf.uint8) # We need to convert it back to unint8 to display it properly

image_label    = tf.string_split([image_path] , delimiter=os.path.sep).values[1]

## 3. Perform Image Augmentation

### 3.1 Define an Image Augmentation Function

In [115]:
def augment_image(image):
    
    ### GAMMA SHIFTING => It affects primarily the high lights ###
    
    random_gamma      = tf.random_uniform([], 0.5, 1.1)
    image_aug         = image ** random_gamma
    
    ### BRIGHTNESS SHIFTING ###
    
    # This gives a centered random  image*(1 +/- delta)
    # It does not fit our requirements, we would like a random brightness not centered around "1".
    #image = tf.image.random_brightness(image, max_delta=0.125) 
    
    random_brightness = tf.random_uniform([], 0.5, 1.2)
    image         =  image * random_brightness
    
    ### OPS SHIFTING ###   
    
    image = tf.image.random_saturation(image, lower=0.5, upper=1.5)
    image = tf.image.random_hue(image, max_delta=0.2)
    image = tf.image.random_contrast(image, lower=0.5, upper=1.5)
    
    # randomly horizontally flip the image
    do_flip = tf.random_uniform([], 0, 1)
    image  = tf.cond(do_flip > 0.5, lambda: tf.image.flip_left_right(image), lambda: image)
    
    # randomly rotate the image
    n_rot = tf.random_uniform([], 0, 3, tf.int32) # 0 => No Rotation, 1 => 90° Rot, 2 => 180° Rot, 3 => 270° Rotation
    image = tf.image.rot90(image, n_rot)
    
     # The random_* ops do not necessarily clamp.
    image = tf.clip_by_value(image, 0.0, 255.0)
    
    return tf.cast(image, tf.uint8)

### 3.2. Create a Tensor of Images

In [116]:
img_arr = tf.stack([
    tf.image.encode_jpeg(image_data),
])

for _ in range(30):
    img_arr = tf.concat([img_arr, [tf.image.encode_jpeg(augment_image(image_resized))]], 0)

## 4. Define an Initialisation Operation

In [117]:
init_op_global = tf.global_variables_initializer()
init_op_local = tf.local_variables_initializer()

## 5. Define a function generating random filenames

In [118]:
def id_generator(size=20, chars=string.ascii_uppercase + string.ascii_lowercase + string.digits):
    return ''.join(random.choice(chars) for _ in range(size))

## 5. Launch the dataset generation Session

In [121]:
with tf.Session() as sess:
    sess.run([init_op_global, init_op_local])

    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    try:
        i = 0
        while not coord.should_stop():
            
            _lbl_txt, _img_arr = sess.run([image_label, img_arr])   
            
            ## Increment ops count
            i += 1 
            
            out_dir = OUTPUT_DIRECTORY + "/" + _lbl_txt.decode("utf-8")
            
            if not os.path.exists(out_dir):
                os.makedirs(out_dir)
                 
            for _img in _img_arr:
                filename = out_dir + "/" + id_generator() + ".jpg"

                with open(filename, "wb+") as f:
                    f.write(_img)
                    f.close()
            
            if (i % 300 == 0):
                print ("Processing Image:", i)
            
    except tf.errors.OutOfRangeError:
        pass
    
    finally:        
        print("\nNumber of Images Processed:", i)
        
        coord.request_stop()
        coord.join(threads)

Processing Image: 300
Processing Image: 600
Processing Image: 900
Processing Image: 1200
Processing Image: 1500
Processing Image: 1800
Processing Image: 2100
Processing Image: 2400
Processing Image: 2700
Processing Image: 3000
Processing Image: 3300
Processing Image: 3600

Number of Images Processed: 3670
